## Эмпирическая функция распределения. Поведение в точке.

### Цель работы:
- ознакомится с определением ЭФР и ее поведением при фиксированном значении аргумента
- аналитически и графически оценить надежность асимптотического интервала
- убедиться в том, что асимптотические методы работают при конечном объеме выборки

## Задание и ход работы:
 
- Выбрать параметры 2-х из 3-х распределений генеральной совокупности $X$:

> $X$ ~ $U(a, b)$
>
> $X$ ~ $e^u$
>
> $X$ ~ $N(a, \sigma ^2)$ 

- Выбрать такую точку $t_0$, что $ 0.05 < F_X(t_0) < 0.95$. Вычислить $F_X(t_0)$

- Смоделировать $m = 10^2$ выборок объема $n = 10^4$ для каждого из двух выбранных распределений.
- Для каждой выборки построить $F_n(t_0)$ - значение эмпирической функции распределения в точке $t_0$ - оценку значения функции распределения в точке $t_0$, то есть величины $F_X(t_0)$.
- Для каждого из распределений получите 100 оценок величины $F_X(t_0)$.

In [37]:
pkg load statistics

In [102]:
m = 10^2;
n = 10^4;
t0 = 0.5;

In [131]:
% For normal disctribution
mu = 0;
sigma = 1;

%     SE = sigma / sqrt(n)
%     F(x) - empirical distribution function(EDF) => sigma^2 = F(t0) * (1 - F(t0))

function normal_distribution(gamma)
    global m n t0 mu sigma
    T = norminv((1 + gamma) / 2);
    X = normrnd(mu, sigma, n, m);
    FX = normcdf(t0, mu, sigma);
    SE = T * sqrt(FX) * (1 - FX) / sqrt(n);
    FX_t0 = mean(X < t0);
    I = [FX_t0 - SE, FX_t0 + SE];
%     ...

endfunction

In [132]:
normal_distribution(0.95)